This section parses a data file containing numeric data gaps represented by NaN. Singular gaps are interpolated as the average of the previous and next rows of data, if both exist; if either is NaN, interpolation is skipped. A new column, `Interpolated`, is used to track interpolations. The column name of the interpolated cell is added to this field, comma-separated if multiple interpolations occur in the same row. After all interpolations, any remaining NaN is replaced with a default value (such as 0). Finally, a new file is written.  
  
As written, this notebook iterates over sequential columns (and assumes a `Date` column). If the columns to check are not sequential, the code will need to be adjusted.

In [10]:
import pandas as pd
import numpy as np

In [ ]:
input_file_path = "../Data/home_value_trim_and_format.csv"
output_file_path = "../Data/home_value_interpolated.csv"
default_value = 0
verbose = False

data_df = pd.read_csv(input_file_path)
data_df.head()

,Date,Year,Month,United States,Alabama,Alaska,Arizona,Arkansas,California,Colorado,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,2004-01,2004,1,161415.797093,112468.897763,187235.313336,166981.743111,101330.145359,320965.660495,222891.848203,...,120966.245893,122685.992092,120843.533394,180548.766033,159287.338362,177558.177299,212860.778161,86900.603167,150926.440573,154013.426255
1,2004-02,2004,2,162463.870016,112725.088423,187841.549097,167847.418906,101783.232071,325732.847350,223182.101684,...,121415.468933,123007.298345,121661.051633,180839.208311,160645.635537,179308.464022,214128.636011,87419.094513,151705.824485,154493.152088
2,2004-03,2004,3,163620.161222,113012.039266,188271.790840,168900.513316,102304.179582,331170.356031,223520.345262,...,122002.587018,123363.249017,122955.182386,181171.778189,162105.970945,181345.555866,215461.768858,88034.036663,152507.819145,155126.857142
3,2004-04,2004,4,164912.797589,113305.182222,189382.536002,170180.251404,102880.788798,337412.489527,224019.582683,...,122502.409568,123763.276144,124239.434257,181569.731315,163691.500174,183784.251765,217005.863589,88581.017025,153457.607735,155979.327633
4,2004-05,2004,5,166368.365614,113666.681304,190588.147633,171738.808920,103421.568193,344599.782061,224670.441514,...,123091.200481,124167.724328,125300.686962,181787.254249,165682.582390,186704.735618,218988.123794,89089.913281,154662.247862,157153.052502


In [12]:
# Show which columns have any data gaps. This is for informational purposes only.

gaps_df = data_df.loc[:,data_df.isna().any()]
gaps_df

,Alaska,Arizona,Idaho,Montana,North Dakota,South Dakota,West Virginia
0,187235.313336,166981.743111,136881.879472,NaN,NaN,120966.245893,86900.603167
1,187841.549097,167847.418906,137349.428041,NaN,NaN,121415.468933,87419.094513
2,188271.790840,168900.513316,137889.972355,NaN,NaN,122002.587018,88034.036663
3,189382.536002,170180.251404,138516.947658,NaN,NaN,122502.409568,88581.017025
4,190588.147633,171738.808920,139213.932465,NaN,NaN,123091.200481,89089.913281
...,...,...,...,...,...,...,...
224,359570.006602,450909.855503,474756.079643,453117.435994,257457.034168,297511.632608,154831.425791
225,358789.792135,444441.126123,467660.143350,449395.467038,257544.312457,297143.781578,154627.641532
226,357870.882403,437424.716679,460962.385642,446428.053405,257834.580429,297103.057878,154434.551756
227,357246.882426,430528.770336,454754.411709,443823.773480,257893.230327,297208.520466,154661.824257


In [13]:
# Add Interpolated column to original dataframe. This column will contain the column names where data was interpolated for any given row.

data_df['Interpolated'] = ""
data_df

,Date,Year,Month,United States,Alabama,Alaska,Arizona,Arkansas,California,Colorado,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Interpolated
0,2004-01,2004,1,161415.797093,112468.897763,187235.313336,166981.743111,101330.145359,320965.660495,222891.848203,...,122685.992092,120843.533394,180548.766033,159287.338362,177558.177299,212860.778161,86900.603167,150926.440573,154013.426255,
1,2004-02,2004,2,162463.870016,112725.088423,187841.549097,167847.418906,101783.232071,325732.847350,223182.101684,...,123007.298345,121661.051633,180839.208311,160645.635537,179308.464022,214128.636011,87419.094513,151705.824485,154493.152088,
2,2004-03,2004,3,163620.161222,113012.039266,188271.790840,168900.513316,102304.179582,331170.356031,223520.345262,...,123363.249017,122955.182386,181171.778189,162105.970945,181345.555866,215461.768858,88034.036663,152507.819145,155126.857142,
3,2004-04,2004,4,164912.797589,113305.182222,189382.536002,170180.251404,102880.788798,337412.489527,224019.582683,...,123763.276144,124239.434257,181569.731315,163691.500174,183784.251765,217005.863589,88581.017025,153457.607735,155979.327633,
4,2004-05,2004,5,166368.365614,113666.681304,190588.147633,171738.808920,103421.568193,344599.782061,224670.441514,...,124167.724328,125300.686962,181787.254249,165682.582390,186704.735618,218988.123794,89089.913281,154662.247862,157153.052502,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,2022-09,2022,9,346524.037497,221833.821012,359570.006602,450909.855503,197718.901413,746931.891976,554152.650140,...,306082.547824,307161.115290,533392.544236,370578.720697,361388.220706,586096.817679,154831.425791,275925.211317,332745.853127,
225,2022-10,2022,10,345296.962654,221508.269570,358789.792135,444441.126123,197528.856407,740263.699280,549260.627844,...,304915.930013,306211.183227,526692.532855,369381.295394,360563.503409,579385.486153,154627.641532,275333.809269,333095.320805,
226,2022-11,2022,11,344289.471095,220962.332115,357870.882403,437424.716679,197244.778127,735023.432331,545491.892457,...,303676.170850,305209.158422,521271.741133,369015.072114,360324.952260,574864.664814,154434.551756,275322.100276,333812.582534,
227,2022-12,2022,12,343374.431911,220464.728518,357246.882426,430528.770336,196869.760669,730531.584283,542291.714419,...,302464.068315,303921.620903,516939.675556,368842.991700,360449.283339,571473.103378,154661.824257,275653.176755,334299.301405,


In [ ]:
# Iterate through the rows to find NaN cells.
# If the previous and next rows for that column contain data, replace NaN with the average and add the column name to the Interpolated column.
# Otherwise, replace NaN with default_value.
# This assumes a numeric index.

start_column = 3
end_column = 54
last_row = len(data_df)

for index, row in data_df.iterrows():
    for column in range(start_column, end_column + 1):
        if data_df.isna().iloc[index, column]:
            # never attempt to interpolate the first or last row
            if (index > 0) & (index < last_row):
                # check previous and next month for NaN
                if not (data_df.isna().iloc[index - 1, column] | data_df.isna().iloc[index + 1, column]):
                    interpolated_value = (data_df.iloc[index - 1, column] + data_df.iloc[index + 1, column]) / 2
                    data_df.iloc[index, column] = interpolated_value
                    print(f"{data_df.columns[column]}, {data_df.iloc[index, 0]}: Interpolating between {data_df.iloc[index - 1, column]} and {data_df.iloc[index + 1, column]} = {interpolated_value}")
                    # add column name to Interpolated column
                    if data_df.loc[index, "Interpolated"] != "":
                        data_df.loc[index, "Interpolated"] += ","
                    data_df.loc[index, "Interpolated"] += data_df.columns[column]
                else:
                    if verbose:
                        print(f"{data_df.columns[column]}, {data_df.iloc[index, 0]}: Skipping interpolation. Replacing NaN with {default_value}.")

# Replace remaining NaN with default_value
data_df = data_df.replace(np.nan, default_value)

Arizona, 2004-07: Interpolating between 173485.4046392401 and 178257.00222767697 = 175871.20343345852
Idaho, 2005-10: Interpolating between 159775.65765068537 and 169519.5739236401 = 164647.61578716274
West Virginia, 2008-10: Interpolating between 106305.04533289708 and 104498.94007512736 = 105401.99270401223
South Dakota, 2012-05: Interpolating between 148649.8882712033 and 151541.35479330344 = 150095.62153225337
Alaska, 2019-03: Interpolating between 293084.37463581417 and 301728.2591689301 = 297406.3169023722


In [15]:
# Verify no more NaN
data_df.loc[:,data_df.isna().any()]

""
0
1
2
3
4
...
224
225
226
227


In [16]:
# Show the interpolated data points
interpolated_data_points = data_df[data_df['Interpolated'] != ""][["Date", "Interpolated"]]
print(interpolated_data_points)
print(f"\nCount = {len(interpolated_data_points)}")

        Date   Interpolated
6    2004-07        Arizona
21   2005-10          Idaho
57   2008-10  West Virginia
100  2012-05   South Dakota
182  2019-03         Alaska

Count = 5


In [17]:
# Output interpolated file

data_df.to_csv(output_file_path, index=False)